In [8]:
# USER OPTIONS
# define tokenizer name - should be one among the keys in the cell below
tokenizer_name = 'ChordSymbolTokenizer' # or any other name from the keys in tokenizers dictionary
# tokenizer_name = 'RootTypeTokenizer'
# tokenizer_name = 'PitchClassTokenizer'
# folder to xmls
val_dir = '/media/maindisk/maximos/data/hooktheory_test'
# val_dir = '/media/maindisk/maximos/data/gjt_melodies/Library_melodies'
# val_dir = '/media/datadisk/datasets/gjt_melodies/Library_melodies'
# val_dir = '/media/maximos/9C33-6BBD/data/gjt_melodies/Library_melodies'

# define batch size depending on GPU availability / status
batchsize = 16
# select device name - could be 'cpu', 'cuda', 'coda:0', 'cuda:1'...
device_name = 'cuda'

In [9]:
import pickle
with open('data/idioms_mir_quick_reference.pickle', 'rb') as f:
    idioms = pickle.load(f)

In [65]:
bc_major = idioms['BachChorales_[0 2 4 5 7 9 11]']
bc_minor = idioms['BachChorales_[0 2 3 5 7 8 10]']
jazz_maj = idioms['Jazz_[0 2 4 5 7 9 11]']
jazz_min = idioms['Jazz_[0 2 3 5 7 9 10]']
organum = idioms['organum_[0 2 3 5 7 8 10]']

In [11]:
from data_utils import SeparatedMelHarmMarkovDataset
import os
import numpy as np
from harmony_tokenizers_m21 import ChordSymbolTokenizer, RootTypeTokenizer, \
    PitchClassTokenizer, RootPCTokenizer, GCTRootPCTokenizer, \
    GCTSymbolTokenizer, GCTRootTypeTokenizer, MelodyPitchTokenizer, \
    MergedMelHarmTokenizer
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import BartForConditionalGeneration, BartConfig, DataCollatorForSeq2Seq
from tqdm import tqdm
from models import TransGraphVAE
import csv

/home/maximos/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
tokenizers = {
    'ChordSymbolTokenizer': ChordSymbolTokenizer,
    'RootTypeTokenizer': RootTypeTokenizer,
    'PitchClassTokenizer': PitchClassTokenizer,
    'RootPCTokenizer': RootPCTokenizer,
    'GCTRootPCTokenizer': GCTRootPCTokenizer,
    'GCTSymbolTokenizer': GCTSymbolTokenizer,
    'GCTRootTypeTokenizer': GCTRootTypeTokenizer
}

In [13]:
melody_tokenizer = MelodyPitchTokenizer.from_pretrained('saved_tokenizers/MelodyPitchTokenizer')
harmony_tokenizer = tokenizers[tokenizer_name].from_pretrained('saved_tokenizers/' + tokenizer_name)

tokenizer = MergedMelHarmTokenizer(melody_tokenizer, harmony_tokenizer)

bart_path = 'saved_models/bart/' + tokenizer_name + '/' + tokenizer_name + '.pt'

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
bart_config = BartConfig(
    vocab_size=len(tokenizer.vocab),
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    decoder_start_token_id=tokenizer.bos_token_id,
    forced_eos_token_id=tokenizer.eos_token_id,
    max_position_embeddings=512,
    encoder_layers=8,
    encoder_attention_heads=8,
    encoder_ffn_dim=512,
    decoder_layers=8,
    decoder_attention_heads=8,
    decoder_ffn_dim=512,
    d_model=512,
    encoder_layerdrop=0.3,
    decoder_layerdrop=0.3,
    dropout=0.3
)

bart = BartForConditionalGeneration(bart_config)

if torch.cuda.is_available():
    checkpoint = torch.load(bart_path, weights_only=True)
else:
    checkpoint = torch.load(bart_path, map_location="cpu", weights_only=True)
bart.load_state_dict(checkpoint)

/home/maximos/.local/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


<All keys matched successfully>

In [16]:
enc1 = bart.model.encoder
enc2 = bart.get_encoder()
dec1 = bart.model.decoder
dec2 = bart.get_decoder()

In [17]:
dec1 == dec2

True

In [18]:
test_dir = '/mnt/ssd2/maximos/data/hooktheory_test'
test_dataset = SeparatedMelHarmMarkovDataset(test_dir, tokenizer, max_length=512, num_bars=64)

# Data collator for BART
def create_data_collator(tokenizer, model):
    return DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)
# end create_data_collator

In [19]:
collator = create_data_collator(tokenizer, model=bart)
valloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=collator)

In [20]:
model_path = 'saved_models/bart_cvae/' + tokenizer_name + '/' + tokenizer_name + '.pt'

config = {
    'hidden_dim_LSTM': 1024,
    'hidden_dim_GNN': 1024,
    'latent_dim': 1024,
    'condition_dim': 1024,
    'use_attention': False
}

model = TransGraphVAE(transformer=bart, device=device, tokenizer=tokenizer, **config)

if torch.cuda.is_available():
    checkpoint = torch.load(model_path, weights_only=True)
else:
    checkpoint = torch.load(model_path, map_location="cpu", weights_only=True)
model.load_state_dict(checkpoint)

model.to(device)
model.eval()

TransGraphVAE(
  (transformer): BartForConditionalGeneration(
    (model): BartModel(
      (shared): BartScaledWordEmbedding(545, 512, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): BartScaledWordEmbedding(545, 512, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(514, 512)
        (layers): ModuleList(
          (0-7): 8 x BartEncoderLayer(
            (self_attn): BartSdpaAttention(
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=512, out_features=512, bias=True)
            (fc2): Linear(in_feat

In [33]:
batch = next(iter(valloader))

/home/maximos/.local/lib/python3.11/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


In [84]:
with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    # transitions = batch['transitions'].to(device)
    # transitions = torch.tensor(bc_major).reshape(1,bc_major.shape[0], bc_major.shape[1]).to(device)
    # transitions = torch.tensor(bc_minor).reshape(1,bc_minor.shape[0], bc_minor.shape[1]).to(device)
    # transitions = torch.tensor(organum).reshape(1,organum.shape[0], organum.shape[1]).to(device)
    # transitions = torch.tensor(jazz_maj).reshape(1,jazz_maj.shape[0], jazz_maj.shape[1]).to(device)
    transitions = torch.tensor(jazz_min).reshape(1,jazz_min.shape[0], jazz_min.shape[1]).to(device)
    # attention_mask = batch['attention_mask'].to(device)
    attention_mask = None
    output_tokens = []
    output_recon_tokens = []
    print(input_ids)
    print(transitions)
    outputs = model(input_ids, transitions, encoder_attention=attention_mask, generate_max_tokens=500)
    for i in outputs['generated_ids'][0]:
        output_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
    for i in outputs['generated_recon_ids'][0]:
        output_recon_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )

tensor([[  2,   6, 183,  95,   4, 119,  50, 135,  50,   6,  95,  50, 111,  50,
         135,  50,   6,  95,  50, 107,  48, 123,  48, 127,  46,   6,  95,  48,
         111,  50, 127,  46,   6,  95,  48, 111,  50, 127,  46,   6,  95,  50,
         127,  50, 139,  51,   6,  95,  51,   6,  95,   4, 119,  51,   6,  95,
          51, 111,  50, 127,  48,   6,  95,  50,   6,  95,   4, 111,  48, 127,
          48, 131,  50,   6,  95,  50, 103,  48, 119,  46, 131,  45,   6,  95,
          45, 111,  46,   6,  95,  46,   6,  95,  46]], device='cuda:0')
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0',
       dtype=torch.float64)
recon generation
decoder_input_ids: tensor([[2]], device='cuda:0')
normal generation
decoder_input_ids: tensor([[2]], device='cuda:0')


In [85]:
print(output_tokens)

['<s>', '<h>', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '</s>']


In [86]:
print(output_recon_tokens)

['<s>', '<h>', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'G:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'G:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'E:min', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'G:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'C:maj', '</s>']


In [87]:
input_ids = batch['input_ids'].to(device)
print(input_ids)
input_tokens = []
for i in input_ids[0]:
    input_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
print(input_tokens)

tensor([[  2,   6, 183,  95,   4, 119,  50, 135,  50,   6,  95,  50, 111,  50,
         135,  50,   6,  95,  50, 107,  48, 123,  48, 127,  46,   6,  95,  48,
         111,  50, 127,  46,   6,  95,  48, 111,  50, 127,  46,   6,  95,  50,
         127,  50, 139,  51,   6,  95,  51,   6,  95,   4, 119,  51,   6,  95,
          51, 111,  50, 127,  48,   6,  95,  50,   6,  95,   4, 111,  48, 127,
          48, 131,  50,   6,  95,  50, 103,  48, 119,  46, 131,  45,   6,  95,
          45, 111,  46,   6,  95,  46,   6,  95,  46]], device='cuda:0')
['<s>', '<bar>', 'ts_6x4', 'position_0x00', '<rest>', 'position_3x00', 'P:64', 'position_5x00', 'P:64', '<bar>', 'position_0x00', 'P:64', 'position_2x00', 'P:64', 'position_5x00', 'P:64', '<bar>', 'position_0x00', 'P:64', 'position_1x50', 'P:62', 'position_3x50', 'P:62', 'position_4x00', 'P:60', '<bar>', 'position_0x00', 'P:62', 'position_2x00', 'P:64', 'position_4x00', 'P:60', '<bar>', 'position_0x00', 'P:62', 'position_2x00', 'P:64', 'position_4x0

In [88]:
bart_outputs = bart.generate(
    input_ids=input_ids,
    # attention_mask=batch['attention_mask'][bi],
    bos_token_id=tokenizer.vocab[tokenizer.harmony_tokenizer.start_harmony_token],
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=500,
    do_sample=True,
    temperature=1.0
)

In [89]:
print(bart_outputs)
bart_only_outputs = []
for i in bart_outputs[0]:
    bart_only_outputs.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
print(bart_only_outputs)

tensor([[  2, 196,   6,  95, 197,   6,  95, 197,   6,  95, 197,   6,  95, 197,
           6,  95, 459,   6,  95, 459,   6,  95, 258,   6,  95, 258,   6,  95,
         197,   6,  95, 197,   6,  95, 197,   6,  95, 197,   6,  95, 459,   6,
          95, 459,   6,  95, 258,   6,  95, 258,   3]], device='cuda:0')
['<s>', '<h>', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'D:dim', '<bar>', 'position_0x00', 'D:dim', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'D:dim', '<bar>', 'position_0x00', 'D:dim', '</s>']


In [90]:
print(bart_only_outputs)
print(output_tokens)
print(output_recon_tokens)

['<s>', '<h>', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'D:dim', '<bar>', 'position_0x00', 'D:dim', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'D:dim', '<bar>', 'position_0x00', 'D:dim', '</s>']
['<s>', '<h>', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', '</s>']
['<s>', '<h>', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'G:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'G:

In [91]:
tokenizer.decode( input_tokens + bart_only_outputs[1:], output_format='file', output_path='examples/bart.mxl' )
tokenizer.decode( input_tokens + output_tokens[1:], output_format='file', output_path='examples/encdec.mxl' )
tokenizer.decode( input_tokens + output_recon_tokens[1:], output_format='file', output_path='examples/recon.mxl' )

Saved as examples/bart.mxl
Saved as examples/encdec.mxl
Saved as examples/recon.mxl


In [92]:
print( (outputs['generated_markov']-transitions.to('cpu')).pow(2).sum().sqrt() )
print( (outputs['recon_markov']-transitions.to('cpu')).pow(2).sum().sqrt() )

tensor(17.4805, dtype=torch.float64)
tensor(7.6245, dtype=torch.float64)
